In [362]:
import pandas as pd 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as pyplot

Data Preprocessing

In [363]:
data = pd.read_csv('payment-practices.csv')

In [372]:
data.head()

,Average time to pay,% Invoices paid within 30 days,% Invoices paid between 31 and 60 days,% Invoices paid later than 60 days,% Invoices not paid within agreed terms,Shortest (or only) standard payment period,Longest standard payment period,Maximum contractual payment period,Payment terms have changed,Suppliers notified of changes,Participates in payment codes,E-Invoicing offered,Supply-chain financing offered,Policy covers charges for remaining on supplier list,Charges have been made for remaining on supplier list
0,25.0,77.0,20.0,3.0,89.0,30.0,60.000000,60.0,False,True,False,True,False,False,False
1,69.0,7.0,28.0,65.0,0.0,30.0,70.820073,75.0,False,True,False,False,False,False,False
2,21.0,89.0,9.0,2.0,47.0,2.0,30.000000,30.0,False,True,False,False,False,True,True
3,14.0,90.0,8.0,2.0,31.0,2.0,30.000000,30.0,False,True,False,False,False,True,True
4,15.0,96.0,2.0,2.0,14.0,2.0,30.000000,30.0,False,False,False,False,False,True,False


In [373]:
data.describe()

,Average time to pay,% Invoices paid within 30 days,% Invoices paid between 31 and 60 days,% Invoices paid later than 60 days,% Invoices not paid within agreed terms,Shortest (or only) standard payment period,Longest standard payment period,Maximum contractual payment period
count,77681.000000,77681.000000,77681.000000,77681.000000,77681.000000,77681.000000,77681.000000,77681.000000
mean,36.681300,55.978029,30.119553,13.896134,27.819875,20.522241,70.820073,73.439266
std,21.970824,27.176096,19.660205,16.004926,22.651489,20.650062,36.063737,82.300103
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,25.000000,34.000000,14.000000,3.000000,10.000000,3.000000,60.000000,45.000000
50%,36.000000,55.978029,30.119553,9.000000,25.000000,20.522241,70.820073,60.000000
75%,45.000000,79.000000,43.000000,17.000000,39.000000,30.000000,70.820073,90.000000
max,1120.000000,100.000000,100.000000,100.000000,100.000000,1000.000000,1264.000000,5475.000000


In [364]:
data.drop(columns=['Report Id','Start date', 'End date', 'Filing date', 'Company',
       'Company number','URL','Payments made in the reporting period'],axis=1,inplace=True)

In [365]:
missing_data = pd.DataFrame({'total_missing': data.isnull().sum(), 'perc_missing': (data.isnull().mean())*100})
missing_data

,total_missing,perc_missing
Average time to pay,6859,8.829701
% Invoices paid within 30 days,6859,8.829701
% Invoices paid between 31 and 60 days,6859,8.829701
% Invoices paid later than 60 days,6859,8.829701
% Invoices not paid within agreed terms,6708,8.635316
Shortest (or only) standard payment period,6708,8.635316
Longest standard payment period,30562,39.342954
Maximum contractual payment period,6708,8.635316
Payment terms have changed,6708,8.635316
Suppliers notified of changes,76016,97.856619


In [366]:
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = ['Payment_terms_have_changed', 'Suppliers_notified_of_changes','Participates in payment codes' , 'E-Invoicing_offered', 'Supply-chain_financing_offered','Policy covers charges for remaining on supplier list','Charges have been made for remaining on supplier list']
data = data.astype({'Participates in payment codes':'bool'})

In [367]:
for var in numerical_cols:
    data[var].fillna(data[var].mean(), inplace=True)

C:\Users\krish\AppData\Local\Temp\ipykernel_11116\797122091.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[var].fillna(data[var].mean(), inplace=True)


In [368]:
missing_data = pd.DataFrame({'total_missing': data.isnull().sum(), 'perc_missing': (data.isnull().mean())*100})
missing_data

,total_missing,perc_missing
Average time to pay,0,0.000000
% Invoices paid within 30 days,0,0.000000
% Invoices paid between 31 and 60 days,0,0.000000
% Invoices paid later than 60 days,0,0.000000
% Invoices not paid within agreed terms,0,0.000000
Shortest (or only) standard payment period,0,0.000000
Longest standard payment period,0,0.000000
Maximum contractual payment period,0,0.000000
Payment terms have changed,6708,8.635316
Suppliers notified of changes,76016,97.856619


In [369]:
missing_data_cols = ['Payment terms have changed', 'Suppliers notified of changes', 'E-Invoicing offered', 'Supply-chain financing offered','Policy covers charges for remaining on supplier list','Charges have been made for remaining on supplier list']

In [370]:
for i in missing_data_cols:
    data_complete = data.drop(columns=missing_data_cols,axis=1)
    data_complete[i] = data[i]
    data_incomplete = data_complete[data_complete.isnull().any(axis=1)]
    data_complete.dropna(inplace=True)
    data_complete = data_complete.astype({i:'bool'})
    X_train = data_complete.drop(columns=[i])
    y_train = data_complete[i]
    model = LogisticRegression()
    model.fit(X_train, y_train)
    predicted_values = model.predict(data_incomplete.drop(columns=[i]))
    data_incomplete[i] = predicted_values
    data.update(data_incomplete)

C:\Users\krish\AppData\Local\Temp\ipykernel_11116\229299691.py:13: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.update(data_incomplete)
C:\Users\krish\AppData\Local\Temp\ipykernel_11116\229299691.py:13: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.update(data_incomplete)
C:\Users\kri

In [371]:
print(data.isnull().sum())

Average time to pay                                      0
% Invoices paid within 30 days                           0
% Invoices paid between 31 and 60 days                   0
% Invoices paid later than 60 days                       0
% Invoices not paid within agreed terms                  0
Shortest (or only) standard payment period               0
Longest standard payment period                          0
Maximum contractual payment period                       0
Payment terms have changed                               0
Suppliers notified of changes                            0
Participates in payment codes                            0
E-Invoicing offered                                      0
Supply-chain financing offered                           0
Policy covers charges for remaining on supplier list     0
Charges have been made for remaining on supplier list    0
dtype: int64
